# トピックモデル入門
## 潜在ディリクレ分配法（LDA）

In [1]:
# ライブラリのインポート
% matplotlib inline
from __future__ import print_function

from collections import Counter
import json
import os

import lda
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
# データの読み込み
with open(os.path.normpath('./dataset/document_word_data.json'), 'r') as f:
    doc_data = json.load(f)

all_doc_index = doc_data.keys()
print('Total Documents: ', len(all_doc_index))

Total Documents:  1000


In [3]:
# データの確認
# 715番目の文書に含まれる単語
', '.join(doc_data['715'])

'ママ, 子供, 健康, づくり, 新た, ライフスタイル, 提案, ママ, マルシェ, 府, 府, 市, さまざま, 家族, ら, 日, もん, 商品, ほか, ハロ, 子供, 商品, プレゼント, 各日, 人, 会場, 木, ぬくもり, 木, 子供, づくり, 会, 木, 日, 午後, 時, 今年度, 森林, 林業, 木材, 大使, ミス, 日本, みどり, 帆, 南, さん, 府, 木材, 会, 湯川, 昌子, さん, ら, 女性, 人, 参加, スギ, ヒノキ, 木材, 放出, 健康, 効果, 木材, こと, 女性, 入場, 無料, 文化, 園, 入園, 料, 大人, 円, 円'

In [4]:
# 全単語のリストを作成
all_vocab = []
for v in doc_data.values():
    for vv in v:
        all_vocab.append(vv)

# 重複を消すためにsetしてlistにする
all_vocab = list(set(all_vocab))
vocab_num = len(all_vocab)
print('Vocablary Number: ', vocab_num)

Vocablary Number:  8709


In [5]:
# スライシングして学習データとテスト用データに分けるため、リストをNumpyの配列にしておきます。
all_doc_index_ar = np.array(list(all_doc_index))

# 学習データのサンプル数を決める
train_portion = 0.7
train_num = int(len(all_doc_index_ar) * train_portion)

# 学習データとテスト用データに分ける
np.random.shuffle(all_doc_index_ar)
train_doc_index = all_doc_index_ar[:train_num]
test_doc_index = all_doc_index_ar[train_num:]

In [6]:
# スパース行列
A_train = sparse.lil_matrix((len(train_doc_index), len(all_vocab)), dtype=np.int)
A_test = sparse.lil_matrix((len(test_doc_index), len(all_vocab)), dtype=np.int)

In [7]:
# all_vocabのリストの中で、単語のインデックス番号を取得するためNumpy配列にしておく
all_vocab_ar = np.array(all_vocab)
train_doc_index_ar = np.array(train_doc_index)
test_doc_index_ar = np.array(test_doc_index)

In [8]:
# 学習用
train_total_elements_num = 0
for i in range(len(train_doc_index)):
    doc_idx = train_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        A_train[i, word_idx] = row_data[word]
        train_total_elements_num += 1
print('Train total elements num :', train_total_elements_num)


# テスト用
test_total_elements_num = 0
for i in range(len(test_doc_index)):
    doc_idx = test_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        A_test[i, word_idx] = row_data[word]
        test_total_elements_num += 1
print('Test total elements num :', test_total_elements_num)

Train total elements num : 34205
Test total elements num : 13839


In [9]:
# CountVectorizerを用いてdoc_dataから簡単に疎行列を作ることもできます
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda a: a, analyzer=lambda a: a)
vectorizer.fit(doc_data[idx] for idx in all_doc_index)
A_train = vectorizer.transform(doc_data[idx] for idx in train_doc_index)
A_test = vectorizer.transform(doc_data[idx] for idx in test_doc_index)

In [10]:
# 実際にLDAを適用してみよう (Scikit-learnを使った例）
# n_topics: トピック数を指定
model1 = LatentDirichletAllocation(n_topics=20,
                                   doc_topic_prior=0.001,
                                   topic_word_prior=0.5,
                                   max_iter=5,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=0)

In [11]:
model1.fit(A_train)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.001,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=0.5, total_samples=1000000.0, verbose=0)

In [12]:
normalize_components = model1.components_ / model1.components_.sum(axis=0)

In [13]:
# http://scikit-learn.org/stable/auto_examples/applications/
# topics_extraction_with_nmf_lda.html　より
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic #0:
クラシック 最後 ぷぅ 光宏 ミニチュア プレゼン デカダンス 対極 ディスプレイ ドル リンクス 匙 景気 即効 オノ インフルエンザ 南ア テンポ 拡充 ステンレス

Topic #1:
マンション 勧告 ステインペン 吉原 スパゲティ 今宵 広樹 バングル ハロウィンマエストロ 本数 ラクオリア 橋 もみじ 油彩 サイセイシマス いくつ 授業 地 有無 フラン

Topic #2:
法律 ハコスコ 全勝 なほ とうさん 昆布 浅口 全治 テクニカル 太宰府天満宮 内山 年収 カウント ェクト 使い手 いちど ノベルティ メイワサンピア 外伝 支所

Topic #3:
後日 いろいろ メディカル シトロン クロエ 実機 千尋 ハベル 入場 旗 冷 ワタシプラス 名実 ジグ ガ スロアプリ おと なめらか 別 灰

Topic #4:
ハンデ 損失 恩 桐 橋本 カニサレス 正明 ラヴクラフトクトゥルフ テレコム 入力 付け シット アニメ せっさたくま 円建て フィッシング 心境 当たり年 時差 加入

Topic #5:
振替 カゲ 幕張 ニコン 東宝 スキャンダル 所在地 全国 区分 幼なじみ ディスプレイ ベスト ストック 昨今 学修 ゲリラ リンクトイン 各地 ディスク サンダル

Topic #6:
涼子 岡本 激戦 凡打 岡 ウィンズ ツッコミ 林業 実業 エグジビション 日帰り プラプラ ブックレット カワイイ ラジオ 善悪 東山 出店 もち ヴァルヴァル

Topic #7:
召使い リツトル 匠 ベティ おとな 慎吾 判決 俵山 かげん 勘 滝沢 夫婦 廃棄 味わい ざき 期 チャネル プラザ アントニオ 岳

Topic #8:
周 どっち ライフ ヒョウ カスタマイズ ニラ 方位 亡国 松永 ダルマ 気泡 川澄 困惑 温泉 ラスト 才女 ステッチ 子ども こちら のこぎり

Topic #9:
ピアス 東シナ海 全力 としのり 元手 武 トリ 動機 包括 ゴジエヴァコラボ 岩国 同僚 ワケ 嶋村 ミドル トラディショナル 消化 支援 為替 ニッポン

Topic #10:
政党 マイナス 春奈 フィニッシュ 古巣 宮本 ソフトドリンク スペシャリスト 原因 引き取り手 作り方 ラテックス サウジアラビア 湖 故旧 オシ

In [14]:
doc_topic_data = model1.transform(A_train)
doc_topic_data

array([[  1.63880695e-05,   1.63880695e-05,   1.63880695e-05, ...,
          1.63880695e-05,   1.63880695e-05,   1.63880695e-05],
       [  1.58679784e-05,   1.58679784e-05,   1.58679784e-05, ...,
          1.58679784e-05,   1.58679784e-05,   1.58679784e-05],
       [  8.31946755e-05,   8.31946755e-05,   8.31946755e-05, ...,
          8.31946755e-05,   8.31946755e-05,   8.31946755e-05],
       ..., 
       [  1.13610543e-05,   1.13610543e-05,   1.13610543e-05, ...,
          1.13610543e-05,   1.13610543e-05,   1.13610543e-05],
       [  4.13188993e-06,   4.13188993e-06,   4.13188993e-06, ...,
          4.13188993e-06,   4.13188993e-06,   4.13188993e-06],
       [  5.64907920e-06,   5.64907920e-06,   5.64907920e-06, ...,
          5.64907920e-06,   5.64907920e-06,   5.64907920e-06]])

In [15]:
normalize_doc_topic_data = \
 doc_topic_data / doc_topic_data.sum(axis=1, keepdims=True)

In [16]:
for topic_idx, prob in enumerate(normalize_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

Topic #0: probality: 0.000016
Topic #1: probality: 0.000016
Topic #2: probality: 0.000016
Topic #3: probality: 0.999689
Topic #4: probality: 0.000016
Topic #5: probality: 0.000016
Topic #6: probality: 0.000016
Topic #7: probality: 0.000016
Topic #8: probality: 0.000016
Topic #9: probality: 0.000016
Topic #10: probality: 0.000016
Topic #11: probality: 0.000016
Topic #12: probality: 0.000016
Topic #13: probality: 0.000016
Topic #14: probality: 0.000016
Topic #15: probality: 0.000016
Topic #16: probality: 0.000016
Topic #17: probality: 0.000016
Topic #18: probality: 0.000016
Topic #19: probality: 0.000016


In [17]:
loglikelihood = model1.score(A_test)
ppl = model1.perplexity(A_test)
print('対数尤度: ', loglikelihood)
print('Perplexity: ', ppl)

対数尤度:  -635671.056699
Perplexity:  198431859624.0


In [18]:
test_doc_topic_data = model1.transform(A_test)
normalize_test_doc_topic_data = \
 test_doc_topic_data / test_doc_topic_data.sum(axis=1, keepdims=True)
for topic_idx, prob in enumerate(normalize_test_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

Topic #0: probality: 0.000007
Topic #1: probality: 0.000007
Topic #2: probality: 0.000007
Topic #3: probality: 0.681624
Topic #4: probality: 0.000007
Topic #5: probality: 0.000007
Topic #6: probality: 0.000007
Topic #7: probality: 0.000007
Topic #8: probality: 0.000007
Topic #9: probality: 0.000007
Topic #10: probality: 0.000007
Topic #11: probality: 0.318252
Topic #12: probality: 0.000007
Topic #13: probality: 0.000007
Topic #14: probality: 0.000007
Topic #15: probality: 0.000007
Topic #16: probality: 0.000007
Topic #17: probality: 0.000007
Topic #18: probality: 0.000007
Topic #19: probality: 0.000007


In [21]:
# LDAを適用してみよう (ldaパッケージを使った場合)
model2 = lda.LDA(n_topics=20, n_iter=1500, random_state=1, alpha=0.5, eta=0.5)
model2.fit(A_train)
topic_word = model2.topic_word_
n_top_words = 20
for topic_idx, topic in enumerate(topic_word):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

INFO:lda:n_documents: 700
INFO:lda:vocab_size: 8709
INFO:lda:n_words: 61448
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -701796
INFO:lda:<10> log likelihood: -549960
INFO:lda:<20> log likelihood: -525280
INFO:lda:<30> log likelihood: -518245
INFO:lda:<40> log likelihood: -515623
INFO:lda:<50> log likelihood: -514603
INFO:lda:<60> log likelihood: -514227
INFO:lda:<70> log likelihood: -513453
INFO:lda:<80> log likelihood: -511959
INFO:lda:<90> log likelihood: -511978
INFO:lda:<100> log likelihood: -510700
INFO:lda:<110> log likelihood: -510176
INFO:lda:<120> log likelihood: -510188
INFO:lda:<130> log likelihood: -510407
INFO:lda:<140> log likelihood: -509837
INFO:lda:<150> log likelihood: -510330
INFO:lda:<160> log likelihood: -509882
INFO:lda:<170> log likelihood: -509924
INFO:lda:<180> log likelihood: -510619
INFO:lda:<190> log likelihood: -509971
INFO:lda:<200> log likelihood: -509984
INFO:lda:<210> log likelihood: -510469
INFO:lda:<220> log likelihood: -5

Topic #0:
後日 プロダクション いろいろ 椎名 本意 木漏れ日 ラッピング キュレ タワマン キム コンシリウム 歯ぎしり マンコ 兵団 炉 ナイン キッズプログラム 尾崎 ガッツ そいつ

Topic #1:
名実 ウェイ 安佐北 パイオニア 交差 インド 出所 排尿 グラマラス 報告 冷 深夜 四季 洗い 嗜好 別れ ホコリセンサ 助っ人 国境 デバ

Topic #2:
激戦 さつまいも マギ 斜め 半日 エフエフ サステナブル 宇宙 バック 忍 お勤め アラン 板ずり 土人 てらそま フラッシュ 差し引き 利奈 悪ふざけ リビング

Topic #3:
メディカル いろいろ 後日 サン 愛嬌 なめらか シトロン クロエ 千尋 実機 入場 旗 スキ 冷 ソフビ 大曲 ガ ワタシプラス 彼岸 ジグ

Topic #4:
政党 チャクシンガイッケンアリマス 潤子 先ほど キャスト マイナス バツ モニタ りょう 春 におい 春奈 無人 づくし 工具 たこ 商標 チャンピオンシップ ジワッ 今村

Topic #5:
模倣 サン ルネサンス ナビディスプレイ キムポテサラダ 崇 フィナンシェ 無敵 人妻 モダン ソロ ライブラリ いじり 彼岸 必死 すき 幸子 松嶋 人志 ホン

Topic #6:
ヒビ くだ 喪服 恋文 山口 ラインナップ 日照 焦点 アスクル エドウィン ビジネス 抹消 バエス キンドルアンリミテッド 太平洋 残念 仕様 イモウト 多分 母体

Topic #7:
かに デモ 検出 ヒ オランド アップ 政 接 佐川 つきあい 次 パイプライン 変動 寝室 幸恵 水曜 カミナンド 瀬 極右 アディクション

Topic #8:
イラ 新妻 月曜 民俗 シャンゼリゼ 格 トキメク 南シナ海 ブロゾヴィッチ 峰 ウロコ イラストコラボ グラウンド づくし 慶大 民代 スキ 味付け 妻 そく

Topic #9:
整形 潘 妊婦 ヤ 島内 合掌 居 尽力 ドコ レコチョク 感動 お前 ピット 改ざん イマデカケルタマ バングル デザイン 意 オリジナリティ 咀嚼

Topic #10:
むかご デトックス バンコク 健康 塩分 村民 そいつ とうや オカルト 同右 晃 新た 憲 カシオ レイトランチコン カスタム ドラゴンズ 校 半島 ワイ

In [22]:
model2.loglikelihood()

-509753.1267447145

In [23]:
doc_topic_data2 = model2.transform(A_train)
for topic_idx, prob in enumerate(doc_topic_data2[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))


Topic #0: probality: 0.006239
Topic #1: probality: 0.009452
Topic #2: probality: 0.006699
Topic #3: probality: 0.070968
Topic #4: probality: 0.004390
Topic #5: probality: 0.002607
Topic #6: probality: 0.004913
Topic #7: probality: 0.011187
Topic #8: probality: 0.004820
Topic #9: probality: 0.005931
Topic #10: probality: 0.005462
Topic #11: probality: 0.004817
Topic #12: probality: 0.595482
Topic #13: probality: 0.008055
Topic #14: probality: 0.003563
Topic #15: probality: 0.006330
Topic #16: probality: 0.005445
Topic #17: probality: 0.233463
Topic #18: probality: 0.005778
Topic #19: probality: 0.004400


In [24]:
# （参考）ディリクレ分布の挙動
alpha = 0.1
K = 6
sampled_probs = np.random.dirichlet([alpha for i in range(K)])
for i, prob in enumerate(sampled_probs):
    print('サイコロ %d面  確率: %.2f'%(i+1, prob))

サイコロ 1面  確率: 0.00
サイコロ 2面  確率: 0.18
サイコロ 3面  確率: 0.00
サイコロ 4面  確率: 0.76
サイコロ 5面  確率: 0.03
サイコロ 6面  確率: 0.03
